
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# Transforming Data Using the Medallion Architecture
![medallion_architecture](../Includes/images/medallion_architecture.png)

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:


1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

2. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

   - Click **More** in the drop-down.

   - In the **Attach to an existing compute resource** window, use the first drop-down to select your unique cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

2. Find the triangle icon to the right of your compute cluster name and click it.

3. Wait a few minutes for the cluster to start.

4. Once the cluster is running, complete the steps above to select your cluster.

## Classroom Setup

Run the following cell to configure your working environment for this course.

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course.

In [0]:
%run ../Includes/Classroom-Setup-03

## A. Configure and Explore Your Environment

1. Set the default catalog to **dbacademy** and the schema to your specific schema. Then, view the current catalog and schema.

In [0]:
%sql
USE CATALOG dbacademy;
USE SCHEMA IDENTIFIER(DA.schema_name);

SELECT current_catalog(), current_schema()

2. View the available files in your schema's **myfiles** volume. Confirm that the volume contains two CSV files, **employees.csv** and **employees2.csv**.

In [0]:
spark.sql(f"LIST '/Volumes/dbacademy/{DA.schema_name}/myfiles/' ").display()

## B. Simple Example of the Medallion Architecture

**Objective**: Create a job that can be scheduled to run on a schedule. The pipeline will:

1. Ingest all CSV files from the **myfiles** volume and create a bronze table.
2. Prepare the bronze table by adding new columns and create a silver table.
3. Create a gold aggregated table for consumers.

### BRONZE
**Objective:** Create a table using all of the CSV files in the **myfiles** volume.

1. Execute the cell to perform the following steps:

    - The `DROP TABLE IF EXISTS` statement drops the **current_employees_bronze** table if it already exists (for demonstration purposes).

    - The `CREATE TABLE IF NOT EXISTS` statement creates the Delta table **current_employees_bronze** if it doesn't already exist and defines its columns.

    - The `COPY INTO` statement:
        - Loads all the CSV files from the **myfiles** volume in your schema into the **current_employees_bronze** table.
        - Uses the first row as headers and infers the schema from the CSV files.

    - The final `SELECT` query displays all rows from the **current_employees_bronze** table.


View the results and confirm that the table contains **6 rows** and **4 columns**.


In [0]:
%sql
-- Drop the table if it exists for demonstration purposes
DROP TABLE IF EXISTS current_employees_bronze;

-- Create an empty table and columns
CREATE TABLE IF NOT EXISTS current_employees_bronze (
  ID INT,
  FirstName STRING,
  Country STRING,
  Role STRING
  );

In [0]:
## Create the bronze raw ingestion table and include the CSV file name for the rows
spark.sql(f'''
  COPY INTO current_employees_bronze
  FROM '/Volumes/dbacademy/{DA.schema_name}/myfiles/'
  FILEFORMAT = CSV
  FORMAT_OPTIONS (
    'header' = 'true', 
    'inferSchema' = 'true'
)
''').display()

In [0]:
%sql
SELECT * 
FROM current_employees_bronze;

### SILVER
**Objective**: Transform the bronze table and create the silver table.

1. Create a table named **current_employees_silver** from the **current_employees_bronze** table. 

    The table will:
    - Select the columns **ID**, **FirstName**, **Country**.
    - Convert the **Role** column to uppercase.
    - Add two new columns: **CurrentTimeStamp** and **CurrentDate**.

In [0]:
%sql
-- Create a temporary view to use to merge the data into the final silver table
CREATE OR REPLACE TABLE current_employees_silver AS 
SELECT 
  ID,
  FirstName,
  Country,
  upper(Role) as Role,                 -- Upcase the Role column
  current_timestamp() as CurrentTimeStamp,    -- Get the current datetime
  date(CurrentTimeStamp) as CurrentDate       -- Get the date
FROM current_employees_bronze;

2. View the **current_employees_silver** table. Confirm that the table contains 6 rows and 6 columns.

In [0]:
%sql
SELECT * 
FROM current_employees_silver;

### GOLD
**Objective:** Aggregate the silver table to create the final gold table.

1. Create a temporary view named **temp_view_total_roles** that aggregates the total number of employees by role. Then, display the results of the view.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW temp_view_total_roles AS 
SELECT
  Role, 
  count(*) as TotalEmployees
FROM current_employees_silver
GROUP BY Role;


SELECT *
FROM temp_view_total_roles;

2. Create the final gold table named **total_roles_gold** with the specified columns.

In [0]:
%sql
CREATE TABLE IF NOT EXISTS total_roles_gold (
  Role STRING,
  TotalEmployees INT
);

3. Insert all rows from the aggregated temporary view **temp_view_total_rows** into the **total_roles_gold** table, overwriting the existing data in the table. This overwrites the data in a table but keeps the existing schema and table definition and properties.

    Confirm the following:
    - **num_affected_rows** is *4*
    - **num_inserted_rows** is *4*

In [0]:
%sql
INSERT OVERWRITE TABLE total_roles_gold
SELECT * 
FROM temp_view_total_roles;

4. Query the **total_roles_gold** table to view the total number of employees by role.

In [0]:
%sql
SELECT *
FROM total_roles_gold;

5. View the history of the **total_roles_gold** table.

In [0]:
%sql
DESCRIBE HISTORY total_roles_gold;

## C. Data Governance and Security
**Objectives:** View the lineage of the **total_roles_gold** table and learn how to set its permissions.

### 1. Complete the following to open your schema in the **Catalog Explorer**.

- a. Select the Catalog icon ![catalog_icon](../Includes/images/catalog_icon.png) in the left navigation bar. 

- b. Type the module's catalog name in the search bar (**dbacademy**).

- c. Select the refresh icon ![refresh_icon](../Includes/images/refresh_icon.png) to refresh the **dbacademy** catalog.

- d. Expand the **dbacademy** catalog. Within the catalog, you should see a variety of schemas (databases).

- e. Find and select your schema. You can locate your schema in the setup notes in the first cell. 

- f.  Click the options icon ![options_icon](../Includes/images/options_icon.png) to the right of your schema and choose **Open in Catalog Explorer**.

- g. Notice that the four tables we created in the demo: **current_employees_bronze**, **current_employees_silver**, **total_roles_gold** and **wine_quality_table** are shown in the **Catalog Explorer** for your schema.

- h. In the **Catalog Explorer** select the **total_roles_gold** table.

Leave the **Catalog Explorer** tab open.

### 2. Complete the following to view the **total_roles_gold** table's permissions, history, lineage and insights in Catalog Explorer: 

   a. **Permissions**. 

  - Select the **Permissions** tab. This will display all permissions on the table. Currently the table does not have any permissions set.

  - Select **Grant**. This allows you to add multiple principals and assign privileges to them. Users must have access to the Catalog and Schema of the table.

  - Select **Cancel**. 

   b. **History**

  - Select the **History** tab. This will display the table's history. The **total_roles_gold** table currently has two versions.

   c. **Lineage**

  - Select the **Lineage** tab. This displays the table's lineage. Confirm that the **current_employees_silver** table is shown.

  - Select the **See lineage graph** button ![see_lineage_graph_button](../Includes/images/see_lineage_graph_button.png). This displays the table's lineage visually. You can select the ![plus_button](../Includes/images/plus_button.png) icon to view additional information.

  - Close out of the lineage graph.

   d. **Insights**

  - Select the **Insights** tab. You can use the Insights tab in **Catalog Explorer** to view the most frequent recent queries and users of any table registered in Unity Catalog. The Insights tab reports on frequent queries and user access for the past 30 days.

   e. Close the **Catalog Explorer** browser tab.

##D. Cleanup
1. Drop views and tables.

In [0]:
%sql
-- Drop the tables
DROP TABLE IF EXISTS current_employees_bronze;
DROP TABLE IF EXISTS current_employees_silver;
DROP TABLE IF EXISTS total_roles_gold;

&copy; 2026 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>